# Challenge Kaggle: How Much Did It Rain? II
  - Etudiant: Ghilas BELHADJ
  - Challenge URL: https://www.kaggle.com/c/how-much-did-it-rain-ii

## Description
Il s'agit dans ce challenge de prédire la quantité d'eau de pluie (en mm) recueillit dans des jauges prevu à cet effet sur un interval de temps d'une heure. Pour celà nous disposons de données concernant des précipitations antérieurs. Nous allons dans ce qui suit, essayer d'apprendre un modèle qui puisse faire cette prédiction là.

## Outils

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import cPickle as cpk

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import cross_validation
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

## Données du challenge
[Les données du challenge](https://www.kaggle.com/c/how-much-did-it-rain-ii/data) doivent se trouver dans le dossier `data` afin de pouvoir executer ce IPython Noteook.
```
data
   ├── test.csv
   └── train.csv
```

Les données sont sous formes de séquences groupées par heure, les entrées qui appartiennent à la même heure ont un identifiant commun et unique. Ensuite chaque entrées comprend differentes mesures prise par un radar à un instant donné.


## Description des attributs
On vas se reférer à la [description donnée sur Kaggle](https://www.kaggle.com/c/how-much-did-it-rain-ii/data) pour voir à quoi correspond chaque attribut.

  * Id: Un identifiant unique d'un ensemble d'observations sur une heure
  * minutes_past: La minute à laquelle l'observation a été faite ( relative a l'heure d'observation de chaque ensemble d'observations )
  * radardist_km: Dsitance de la jauge par rapport au radar qui prend la mesure sur cette gauge.
  * Ref: [Réflectivité](https://fr.wikipedia.org/wiki/R%C3%A9flectivit%C3%A9) en km ( c'est un rapport entre l'énergie réfléchie par rapport à l'énergie incidente d'un objet)
  * Ref_5x5_10th: 10émè [centile](https://fr.wikipedia.org/wiki/Centile) de reflexivité dans un voisinage 5x5 autour de la gauge.
  * Ref_5x5_50th: 50ème centile.
  * Ref_5x5_90th: 90ème centile.
  * RefComposite: Reflexivité maximum sur la colonne vertical au dessus de la gauge ( en [décibel Z](https://fr.wikipedia.org/wiki/D%C3%A9cibel_Z) )
  * RefComposite_5x5_10th: 10ème centile.
  * RefComposite_5x5_50th: 50ème centile.
  * RefComposite_5x5_90th: 90ème centile.
  * RhoHV: Un coeficient de correlation (sans unité)
  * RhoHV_5x5_10th: 10ème centile.
  * RhoHV_5x5_50th: 50ème centile.
  * RhoHV_5x5_90th: 90ème centile.
  * Zdr: Reflectivité différentiel en [Décibel](https://fr.wikipedia.org/wiki/D%C3%A9cibel)
  * Zdr_5x5_10th: 10ème centile.
  * Zdr_5x5_50th: 50ème centile.
  * Zdr_5x5_90th: 90ème centile.
  * Kdp: La phase différentielle spécifique (Utilisé pour localiser les régions à forte précipitations/Attenuations)
  * Kdp_5x5_10th: 10ème centile.
  * Kdp_5x5_50th: 50ème centile.
  * Kdp_5x5_90th: 90ème centile.
  * Expected: L'observation actuelle de l'etat de la gauge (en mm)


## Aperçu des données
#### Lecture des données

In [ ]:
data_path = "data/"
train_file= "train.csv"
test_file= "test.csv"

print "Reading %s" % input_file
time0 = dt.datetime.now()

df_train = pd.read_csv(data_path+train_file)
df_train.set_index('Id')

# test_file = pd.read_csv(data_path+train_file)
test_file.set_index('Id')

time1 = dt.datetime.now()
print('Read in %i sec' % (time1-time0).seconds)

In [ ]:
df_train.info()

In [ ]:
df_train.head(10)

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
df_train.corr()

## Analyse des données
Colonne | max | min | std | mean
---|---|---|---|---
minutes_past |  59 | 0 | 17.30 | 29.52
radardist_km |  21 | 0 | 4.20 | 11.06
Ref |  71 | -31 | 10.35 | 22.92
Ref_5x5_10th |  62.5 | -32 | 9.20 | 19.95
Ref_5x5_50th |  69 | -32 | 10.05 | 22.61
Ref_5x5_90th |  72.5 | -28.5 | 11.10 | 25.89
RefComposite |  92.5 | -32 | 10.68 | 24.71
RefComposite_5x5_10th | 66 | -31 | 9.70 | 22.15
RefComposite_5x5_50th | 71 | -27.5 | 10.42 | 24.42
RefComposite_5x5_90th | 93.5 | -25 | 11.49 | 27.36
RhoHV | 1.05 | 0.2 | 0.09 | 0.97
RhoHV_5x5_10th | 1.05 | 0.2 | 0.12 | 0.91
RhoHV_5x5_50th | 1.05 | 0.2 | 0.07 | 0.97
RhoHV_5x5_90th | 1.05 | 0.2 | 0.04 | 1.01
Zdr |   7.93 | -7.87 | 1.51 | 0.53
Zdr_5x5_10th |   7.93 | -7.87 | 1.00 | -0.71
Zdr_5x5_50th |   7.93 | -7.87 | 0.93 | 0.33
Zdr_5x5_90th |   7.93 | -7.87 | 1.67 | 2.07
Kdp | 179 | -96.04 | 3.86 | 0.03
Kdp_5x5_10th |   3.51 | -80.79 | 2.79 | -3.48
Kdp_5x5_50th |  12.8 | -78.77 | 2.26 | -0.47
Kdp_5x5_90th | 144.6 | -100.2 | 4.14 | 4.07
Expected |   33017.73 | 0.01 | 548.60 | 108.62

Une fois que nous avont pris connaissance de chaque attribut et de sa plage de valeur, nous pouvons nettoyer les données des outlayers.

Les attributs dont le max et le min ne permettent pas de retrouver le mean, contienent surrement des outliers.

**Exemple:** *Expected* contient une valeur maximum de 33017.73 ... improbable. Les points qui ont des valeurs plus grande que 102mm/h sont probablement des outliers. (source: http://thewatchers.adorraeli.com/2015/07/15/record-rainfall-freak-storm-dumps-102-mm-of-rain-in-1-hour-norway/)

![Distro de chaque attribut](img/dist.png "Distro de chaque attribut")

## Que faire des valeurs nulles
Certaines données ne sont pas disponible pour une raison ou pour une autre, il est ici question de décider si l'ont doit éliminer les lignes dans lesquelles elle se trouvent, ou bien les déduire afin d'utiliser un plus grand nombre de données.

Les valeurs suivantes représente le nombre de NaN sur chaque colonne:
```
colonne id, 0 NaN
colonne min, 0 NaN
colonne km, 0 NaN
colonne ref, 6533728 NaN
colonne ref10, 7573458 NaN
colonne ref50, 6524131 NaN
colonne ref90, 5368429 NaN
colonne refc, 6176380 NaN
colonne refc10, 7110407 NaN
colonne refc50, 6178189 NaN
colonne refc90, 5102588 NaN
colonne rho, 7928015 NaN
colonne rho10, 8709031 NaN
colonne rho50, 7924578 NaN
colonne rho90, 6986970 NaN
colonne zdr, 7928015 NaN
colonne zdr10, 8709031 NaN
colonne zdr50, 7924578 NaN
colonne zdr90, 6986970 NaN
colonne kdp, 8666078 NaN
colonne kdp10, 9404373 NaN
colonne kdp50, 8660920 NaN
colonne kdp90, 7815206 NaN
colonne exp, 0 NaN
```

Sur les 12 786 237 lignes du fichier ̀̀ train.csv` environs ~7 000 000 valeurs sont manquantes sur chaque colonne. Il s'agit d'un très grand nombre de lignes, et des stratégies doivent être utilisé pour combler ce manque d'information.

Si l'on enlève chaque ligne pour peu qu'elle contienne une valeur nulle, il nous restera 2 735 878 lignes pour l'apprentissage, contre 717 625 à prédir (test.csv).

De plus, on n'as pas le droit de retirer des lignes depuis le fichier `test.csv` car on doit soumettre impérativement 717 625 lignes sur Kaggle pour ce challenge, ce qui implique que l'ont doit remplir les NaN du fichier test.csv pour pouvoir faire de la prédiction.


## Netoyyage des données

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt


data_path = "data/"

input_files=[ "train.csv", "test.csv" ] # 13765202 lines

for input_file in input_files:
    print "Reading %s" % input_file
    time0 = dt.datetime.now()
    df = pd.read_csv(data_path+input_files)
    time1 = dt.datetime.now()
    print('Read in %i sec' % (time1-time0).seconds)

    print """Drop minutes_past columns"""
    df = df.drop('minutes_past', axis=1)

    print """Create sample_weights feature"""
    max_km = df["radardist_km"].max()
    df['sample_weights'] = df['radardist_km'].apply( lambda x:1-x/max_km )

    print """Drop radardist_km columns"""
    df = df.drop('radardist_km', axis=1)

    print """Create missing_values feature"""
    nb_extra_cols = 1
    if ( input_file == "train.csv"):
        nb_extra_cols = 2
    nb_columns = len(df.columns) - nb_extra_cols # without Expected and Id
    df['missing_values'] = df.isnull().sum(axis=1) / nb_columns

    grouped = df.groupby('Id')

    value_cols = [ 'Ref','Ref_5x5_10th','Ref_5x5_50th','Ref_5x5_90th',
    'RefComposite','RefComposite_5x5_10th','RefComposite_5x5_50th','RefComposite_5x5_90th',
    'RhoHV','RhoHV_5x5_10th','RhoHV_5x5_50th','RhoHV_5x5_90th',
    'Zdr','Zdr_5x5_10th','Zdr_5x5_50th','Zdr_5x5_90th',
    'Kdp','Kdp_5x5_10th','Kdp_5x5_50th','Kdp_5x5_90th' ]

    agg_cols = lambda col: {col+'_max' : np.max, col+'_median' : np.median}

    print """Agg by mean"""
    agg_df = grouped.agg(np.mean) 

    print """Create median, max aggregations""" 
    for col in value_cols:
        agg_df = pd.concat( [agg_df, grouped[col].agg(agg_cols(col))], axis=1 )


    print """Fill left NaN with median"""
    agg_df = agg_df.fillna(agg_df.median())


    if ( input_file == "train.csv"):
        print """Clean Expected column"""
        agg_df = agg_df[agg_df['Expected'] <= 100]


    clean_name = "clean_"+input_file
    print """Saving %s\n\n""" % clean_name
    agg_df.to_csv(data_path+clean_name)


## Entrainement des modèles

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import cPickle as cpk

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import cross_validation
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

cleantrain_file="data/clean_train.csv" # 12786237 lines

print "Lecture clean csv..."
time0 = dt.datetime.now()

cols = [ 'Id', 'Expected', 'missing_values',
'Ref_5x5_10th_max','Ref_5x5_50th_max','sample_weights',
'Ref_5x5_90th','RefComposite_5x5_90th','Ref_5x5_90th_max',
'RefComposite_5x5_90th_median','RefComposite_5x5_90th_max',
'Zdr_5x5_90th','Zdr_5x5_90th_max','Ref_5x5_90th_median','Zdr_5x5_90th_median']

df = pd.read_csv(cleantrain_file, usecols=cols, nrows=100000)

time1 = dt.datetime.now()
print('lecture en: %i sec' % (time1-time0).seconds)
print "Nb lignes: %d ",  df.shape[0]

print "Prepare data, labels"
data = df.as_matrix()
label = df[['Expected']].as_matrix().ravel()
df = df.drop(['Expected', 'Id'], axis=1)

# print "Prepare folds for cross validation"
x_train, x_test, y_train, y_test = cross_validation.train_test_split(data, label, test_size=0.8, random_state=23435)


# conf = sklearn.metrics.confusion_matrix(df['missing_values'], df['sample_weights'])
# plt.imshow(conf, cmap='binary', interploation='None')

print "RandomForestRegressor..."
clf = sklearn.ensemble.RandomForestRegressor(verbose=2, n_jobs=2)
clf.fit(x_train, y_train)
print mean_squared_error( clf.predict(x_test), y_test )
# with open('models/RandomForestRegressor.pkl', 'wb') as fid:
#     cpk.dump(clf, fid)

print "GradientBoostingRegressor..."
clf = sklearn.ensemble.GradientBoostingRegressor(verbose=2)
clf.fit(x_train, y_train)
print mean_squared_error( clf.predict(x_test), y_test )
# with open('models/GradientBoostingRegressor.pkl', 'wb') as fid:
#     cpk.dump(clf, fid)

print "ExtraTreesRegressor..."


clf = ExtraTreesRegressor(n_estimators=20, verbose=2, n_jobs=-1)
clf.fit(x_train, y_train)
print mean_squared_error( clf.predict(x_test), y_test )
# with open('models/ExtraTreesRegressor.pkl', 'wb') as fid:
#     cpk.dump(clf, fid)

## Création de la submission

In [ ]:
import cPickle as cpk
import pandas as pd
import datetime as dt
import numpy as np

model_path = "models/"
data_path = "data/"
cleantest_file="clean_test.csv"
models_files = ['GradientBoostingRegressor.pkl', 'RandomForestRegressor.pkl', 'ExtraTreesRegressor.pkl']


cols = [ 'Id', 'missing_values',
'Ref_5x5_10th_max','Ref_5x5_50th_max','sample_weights',
'Ref_5x5_90th','RefComposite_5x5_90th','Ref_5x5_90th_max',
'RefComposite_5x5_90th_median','RefComposite_5x5_90th_max',
'Zdr_5x5_90th','Zdr_5x5_90th_max','Ref_5x5_90th_median','Zdr_5x5_90th_median']

time0 = dt.datetime.now()
df = pd.read_csv(data_path+cleantest_file, usecols=cols)
time1 = dt.datetime.now()
print('lecture en: %i sec' % (time1-time0).seconds)
print "Nb lignes: %d ",  df.shape[0]

print "preparing data"
ids = df['Id']
df = df.drop(['Id'], axis=1)
data = df.as_matrix()


pred_exp = [pd.read_csv("data/marshall-palmer.csv")]

for model_file in models_files:
    with open(model_path+model_file, 'rb') as fid:
        print "Load classifier %s" % model_file
        clf = cpk.load(fid)

        print "Prediction..."
        pred = clf.predict(data)

        print "Saving csv..."
        df_pred = pd.DataFrame({'Expected': pred, 'Id': ids})
        pred_exp.append( pred )

        df_pred.to_csv(data_path+model_file+'_sol.csv', index=False)

sol = pd.DataFrame({'Id': ids, 'Expected': np.mean( pred_exp, axis=0) }) 
solution_file="solution.csv"
sol.to_csv(data_path+solution_file, header=True, cols=["Id","Expected"], index=False)

## Résultats

    * Public  : 24.06898
    * Private : 25.09764